In [1]:
import pandas as pd
import numpy as np

import dask.dataframe as dd

In [3]:
from pathlib import Path

PATH = Path('/data/churn')
list(PATH.iterdir())

[PosixPath('/data/churn/train_v2.csv'),
 PosixPath('/data/churn/user_logs_v2.csv'),
 PosixPath('/data/churn/transactions.csv'),
 PosixPath('/data/churn/WSDMChurnLabeller.scala'),
 PosixPath('/data/churn/user_logs.csv'),
 PosixPath('/data/churn/transactions_v2.csv'),
 PosixPath('/data/churn/members_v3.csv'),
 PosixPath('/data/churn/train.csv'),
 PosixPath('/data/churn/sample_submission_v2.csv'),
 PosixPath('/data/churn/sample_submission_zero.csv')]

In [3]:
!cd /data/churn/ && du -a -h

44M	./train_v2.csv
1.4G	./user_logs_v2.csv
1.7G	./transactions.csv
8.0K	./WSDMChurnLabeller.scala
29G	./user_logs.csv
111M	./transactions_v2.csv
409M	./members_v3.csv
45M	./train.csv
41M	./sample_submission_v2.csv
44M	./sample_submission_zero.csv
33G	.


In [4]:
train = pd.read_csv(PATH/'train_v2.csv')
train.head()

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


In [10]:
members = pd.read_csv(PATH/'members_v3.csv')
members.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [11]:
trans = pd.read_csv(PATH/'transactions.csv')
trans.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,20150930,20151031,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,20150930,20160427,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,20150930,20151128,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,20150930,20151121,0


In [7]:
user_logs = pd.read_csv(PATH/'user_logs_v2.csv', nrows = 1)
user_logs

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,8,4,0,1,21,18,6309.273


In [17]:
with open(PATH/'WSDMChurnLabeller.scala', 'r') as f:
    print(f.read())

import java.time.{LocalDate}
import java.time.format.DateTimeFormatter
import java.time.temporal.ChronoUnit

import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions._

import scala.collection.mutable

object WSDMChurnLabeller {
  def main(args: Array[String]) : Unit ={
    val spark = SparkSession
      .builder()
      .master("local")
      .getOrCreate()

    val data = spark
      .read
      .option("header", value = true)
      .csv("/Users/ardenchiu/wsdm_fix/wsdm_transactions_20170331.csv")

    val historyCutoff = "20170131"

    val historyData = data.filter(col("transaction_date")>="20170101" and col("transaction_date")<=lit(historyCutoff))
    val futureData = data.filter(col("transaction_date") > lit(historyCutoff))

    val calculateLastdayUDF = udf(calculateLastday _)
    val userExpire = historyData
      .groupBy("msno")
      .agg(
        calculateLastdayUDF(
          collect_list(
            struct(
              col("payment_method_id

In [7]:
user_logs = dd.read_csv(PATH/'user_logs_v2.csv')
x = user_logs.count()

In [9]:
x.compute()

msno          18396362
date          18396362
num_25        18396362
num_50        18396362
num_75        18396362
num_985       18396362
num_100       18396362
num_unq       18396362
total_secs    18396362
dtype: int64